# Hyper parameter optimization with hyperopt
## _For MLP model_
## 1. Imports

In [ ]:
# cosmetic imports
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(['dark_background'])
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator, FormatStrFormatter
# real imports
from tflite2xcore.model_generation import utils
import numpy as np
import os
from collections import OrderedDict
from hyperopt import fmin, tpe, hp, Trials, space_eval, plotting
import tensorflow as tf
import joblib
utils.set_all_seeds(42)


## 2. Check that the CPU backend is being used

In [ ]:
tf.config.experimental.set_visible_devices([], 'GPU')
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.get_visible_devices()


## 3. Normalization function
Change the range to (-1, 1) since tanh activation is used in the model. And normalize it statistically.

In [ ]:
def normalize(narray):
    #narray = (2 * narray) - 1
    mean = np.mean(narray)
    std = np.std(narray)
    narray = (narray - mean)/ std
    return narray

## 4. Constants and data for training
Search sapce:
- Learning rate
- Epochs
- Batch size
- Dropout
- L1 regularization

In [ ]:
# Constants
SEARCH_SPACE = OrderedDict([('learning_rate',
                             hp.loguniform('learning_rate', np.log(0.01), np.log(0.5))),
                            ('epochs',
                             hp.choice('epochs', range(10, 71, 1))),
                            ('batch_size',
                             hp.choice('batch_size', [32, 64, 128, 256, 512])),
                            ('dropout',
                            hp.choice('dropout', np.arange(0.1, 0.51, 0.01))),
                            ('l1_reg',
                             hp.choice('l1_reg', np.arange(1e-5, 1e-4, 1e-6)))
                           ])

OUTPUT_DIR = os.path.expanduser('./output/')
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Data
data = utils.prepare_MNIST(False, simard=False, padding=2)
# change range to -1, 1 and normalize for full use of tanh funciton
data['x_train'] = normalize(data['x_train'])
data['x_test'] = normalize(data['x_test'])
data['x_val'] = normalize(data['x_val'])
# average close to zero, minimum below zero
print('Average value of train data: ', np.average(data['x_train']))
print('Minimum value of train data: ', np.min(data['x_train']))
for k, v in data.items():
    print(f"Prepped data[{k}] with shape: {v.shape}")

## 5. Objective function and model declaration

In [ ]:
def train_evaluate(params):
    core_model = tf.keras.Sequential(
        name='hyperopt_test',
        layers=[
            tf.keras.layers.Flatten(input_shape=(32, 32, 1), name='input'),
            tf.keras.layers.Dense(390, activation='tanh', name='dense_1',
                                  kernel_regularizer=tf.keras.regularizers.l1(params['l1_reg'])),
            tf.keras.layers.Dropout(params['dropout']),
            tf.keras.layers.Dense(290, activation='tanh', name='dense_2',
                                 kernel_regularizer=tf.keras.regularizers.l1(params['l1_reg'])),
            tf.keras.layers.Dropout(params['dropout']),
            tf.keras.layers.Dense(10, activation='softmax', name='output',
                                 kernel_regularizer=tf.keras.regularizers.l1(params['l1_reg']))
    ])
    core_model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=params['learning_rate']),
        metrics=['accuracy'])
    core_model.fit(
        data['x_train'], data['y_train'],
        validation_data=(data['x_test'], data['y_test']),
        batch_size=params['batch_size'],
        verbose=0,
        epochs=params['epochs']
    )
    _, accuracy = core_model.evaluate(data['x_test'], data['y_test'])
    return accuracy

def objective(params):
    tf.keras.backend.clear_session()
    all_params = {**params}
    return -1.0 * train_evaluate(all_params)

## 6. Load trials file if it exists

In [ ]:
trials_file = os.path.join(OUTPUT_DIR, 'trials_mlp.pkl')
if os.path.exists(trials_file):
    print('Loading existing trials file ...')
    with open(trials_file, 'rb') as f:
        trials = joblib.load(f)
    start = len(trials.losses())
else:
    start = 0
    trials = Trials()

## 7. Run trials

In [ ]:
MAX_EVALS = 1000
for eval_ind in range(start, MAX_EVALS):
    print('Starting trial {}/{}'.format(eval_ind+1, MAX_EVALS))
    fmin(fn=objective,
         space=SEARCH_SPACE,
         algo=tpe.rand.suggest,
         max_evals=eval_ind+1,
         trials=trials)
    with open(trials_file, 'wb') as f:
        joblib.dump(trials, f)

## 8. Results
### 8.1 Minimum

In [ ]:
# Uncomment below to load trials
#with open(trials_file, 'rb') as f:
#    trials = joblib.load(f)
min = np.min(trials.losses())
print(-min)

In [ ]:
best_trial = {}
for key, val in trials.best_trial['misc']['vals'].items():
    best_trial[key] = val[0]
best_params = space_eval(SEARCH_SPACE, best_trial)
print(f"Found minimum after {MAX_EVALS} trials:")
print(best_params)

### 8.2 Accuracy

In [ ]:
objective(best_params)

### 8.3 Acc range and median (50 trainings)

In [ ]:
l = [-objective(best_params) for e in range(50)]
maxv = np.max(l)
minv = np.min(l)
print(f"Acc range: ({maxv}, {minv})\nAcc median: {np.median(l)}")

## 9. Plotting history and histogram

In [ ]:
plotting.main_plot_history(trials)
plotting.main_plot_histogram(trials)